In [5]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from urllib.parse import unquote
from environs import Env

env = Env()
env.read_env(path=r"C:\Users\JuliaSjöholm-AI23GBG\Documents\GitHub\Databases\.env")  # Läs .env filen

# Server- och databasnamn
server_name = "localhost"
database_name = env.str("database")

# Skapar en anslutningssträng
UID = env.str('UID')  # Använd env för att hämta UID
PWD = env.str('PWD')  # Använd env för att hämta PWD
connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};UID={UID};PWD={PWD}"
url_string = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')  

# Försöker skapa en motor och ansluta till databasen
try:
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to database!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)


Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=JuliaSjoholm;UID=test;PWD=lkjf948393ek2ckv 

Successfully connected to database!


In [6]:
from sqlalchemy import text

search = input("Please enter a book title: ")

print(f"Book Search : {search}\n")
# Frågan anropar stored proceduren istället för en direkt SQL-fråga
query = text("EXEC SearchTitles @SearchString = :search_string")

with engine.connect() as conn:    
    result = conn.execute(query, {"search_string" : search})

    print(f"{'Boktitel:'.ljust(40)}{'Pris (SEK):'.ljust(15)}{'Butik:'.ljust(15)}{'Lagersaldo:'}")
    
    for book in result:
        # Anpassa utskriften till resultatstrukturer som returneras från stored proceduren
        print(f"{book.Titel.ljust(40)}{str(book.Pris).ljust(15)}{book.Butik.ljust(15)}{book.Lagersaldo}")

    print()
    print(f"The Query:\n{query}")


Book Search : ma

Boktitel:                               Pris (SEK):    Butik:         Lagersaldo:
Mamman och den vilda bebin              178.00         Lilla boken    0
Mamman och den vilda bebin              178.00         Mellan boken   0
Mamman och den vilda bebin              178.00         Stora boken    2
Madicken                                136.00         Lilla boken    0
Madicken                                136.00         Mellan boken   5
Madicken                                136.00         Stora boken    25
Max balja                               140.00         Lilla boken    0
Max balja                               140.00         Mellan boken   0
Max balja                               140.00         Stora boken    0
Loranga, Masarin & Dartanjang           180.00         Lilla boken    15

The Query:
EXEC SearchTitles @SearchString = :search_string
